### Optimización de hiperparámetros (Optuna)

<a target="_blank" href="https://colab.research.google.com/github/pglez82/DeepLearningWeb/blob/master/labs/notebooks/Optimizaci%C3%B3n%20de%20hiperparámetros%20(Optuna).ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

En el entrenamiento de redes neuronales profundas, existen una multitud de hiperparámetros que podemos optimizar. Algunos de los más importantes son los siguientes:
- Learning rate
- Momento
- Weight decay
- Cantidad de dropout
- Optimizador utilizado
- Número de capas de la red
- Tamaño de las capas de la red (u otros parámetros de la capa como tamaño del kernel para CNNs, etc)
- Funciones de activación usadas
- etc.

Es muy común que la búsqueda de hiperparámetros sea hecha de manera bastante artesanal, siguiendo la intuición y los conocimientos del científico de datos. Aún así, existe software que nos permite hacer esta búsqueda de manera más **sistemática y automática**. Uno de estos software es Optuna, que es el que veremos en esta práctica. Wandb también tiene su sistema de búsqueda de hiperparámetros.
